Introduction: Safety in schools is a very important requirement and the neighbourhood of the school has a big influence on the school safety.
This project will use Chicago school and crime data from the Course 5 of this certification. School data will be used with FourSquare API to find
the available security( number of Police stations).Depending on the crime counts in the neighbourhood of existing schools,it can be decided if more poice stations are required. For new schools,the crime data analysis and developing a model using machine learning can help with predicting the crime in the neighbourhood in future. The project can be used by any location with the location, school and crime rates data available.This will help the school administration and government bodies to make accurate decisions on which locations are appropriate for new schools as well as which existing school need more security arrangements.


Data Section:  This project will use Chicago school and crime data from the Course 5 of this certification.Only certain columns of interest are 
retrieved from the datasets. The records with missing data in the datasets are dropped. Crime data prior to 2013 is used to develop a linear regression model to be used for predicting crime counts for future years.Chicago location data is obtained from the school data. The location co-ordinates for Chicago is obtained using the geopy.geocoders library Nominatim. Maps for the Chicago school data is displayed. FourSquare API will be used to find the police stations near schools to determine the available security and if additional security is required based on the current crime counts.



Installing and Importing of Libraries

In [51]:
try:
    print("Installing BeautifulSoup4...\n")
    !conda install -c conda-forge beautifulsoup4 --yes
    print("BeautifulSoup4 has been successfully installed!\n")
except:
    print("ERROR: could not install BeautifulSoup4!\n")

try:
    print("Installing ProgressBar...\n")
    !conda install -c conda-forge ProgressBar2 --yes
    print("ProgressBar has been successfully installed!\n")
except:
    print("ERROR: could not install ProgressBar!\n")

try:
    print("Installing lxml...\n")
    !conda install -c conda-forge lxml --yes
    print("lxml has been successfully installed!\n")
except:
    print("ERROR: could not install lxml!\n")

try:
    print("Installing GeoPy...\n")
    !conda install -c conda-forge geopy --yes
    print("GeoPy has been successfully installed!\n")
except:
    print("ERROR: could not install GeoPy!\n")

try:
    print("Installing Folium...\n")
    !conda install -c conda-forge folium=0.5.0 --yes
    print("Folium has been successfully installed!\n")
except:
    print("ERROR: could not install Folium!\n")

try:
    print("Importing libraries...\n")
    from progressbar import ProgressBar
    from bs4 import BeautifulSoup as bts # library for web scraping
    import numpy as np # library to handle data in a vectorized manner
    import pandas as pd # library for data analysis
    from pandas.io.json import json_normalize
    import matplotlib.cm as cm
    import matplotlib.colors as colors
    import requests # library to handle requests
    from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
    import matplotlib as mp # library for visualization
    from sklearn.cluster import KMeans # import k-means from clustering stage
    from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
    import folium # map rendering library
    import lxml
    import re
    from time import sleep
    from sklearn import preprocessing
    from sklearn.model_selection import train_test_split
    from sklearn import linear_model
    from sklearn.metrics import r2_score

    print("All libraries imported successfully!\n")
except:
    print("ERROR: Could not import all libraries!\n")

%matplotlib inline

Installing BeautifulSoup4...

Solving environment: done

# All requested packages already installed.

BeautifulSoup4 has been successfully installed!

Installing ProgressBar...

Solving environment: done

# All requested packages already installed.

ProgressBar has been successfully installed!

Installing lxml...

Solving environment: done

# All requested packages already installed.

lxml has been successfully installed!

Installing GeoPy...

Solving environment: done

# All requested packages already installed.

GeoPy has been successfully installed!

Installing Folium...

Solving environment: done

# All requested packages already installed.

Folium has been successfully installed!

Importing libraries...

All libraries imported successfully!



Retrieving and Cleaning Chicago Crime Data

In [53]:
# Using the link from Course5 for the Chicago crime dataset
chicago_crime_df = pd.read_csv('https://ibm.box.com/shared/static/svflyugsr9zbqy5bmowgswqemfpm1x7f.csv')
print("Number of rows and columns in Chicago crime dataset is: ",chicago_crime_df.shape)
chicago_crime_df.head()

Number of rows and columns in Chicago crime dataset is:  (533, 22)


,ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,...,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,UPDATEDON,LATITUDE,LONGITUDE,LOCATION
0,3512276,HK587712,08/28/2004 05:50:56 PM,047XX S KEDZIE AVE,890,THEFT,FROM BUILDING,SMALL RETAIL STORE,False,False,...,14.0,58.0,6,1155838.0,1873050.0,2004,02/10/2018 03:50:01 PM,41.807441,-87.703956,"(41.8074405, -87.703955849)"
1,3406613,HK456306,06/26/2004 12:40:00 PM,009XX N CENTRAL PARK AVE,820,THEFT,$500 AND UNDER,OTHER,False,False,...,27.0,23.0,6,1152206.0,1906127.0,2004,02/28/2018 03:56:25 PM,41.898280,-87.716406,"(41.898279962, -87.716405505)"
2,8002131,HT233595,04/04/2011 05:45:00 AM,043XX S WABASH AVE,820,THEFT,$500 AND UNDER,NURSING HOME/RETIREMENT HOME,False,False,...,3.0,38.0,6,1177436.0,1876313.0,2011,02/10/2018 03:50:01 PM,41.815933,-87.624642,"(41.815933131, -87.624642127)"
3,7903289,HT133522,12/30/2010 04:30:00 PM,083XX S KINGSTON AVE,840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,...,7.0,46.0,6,1194622.0,1850125.0,2010,02/10/2018 03:50:01 PM,41.743665,-87.562463,"(41.743665322, -87.562462756)"
4,10402076,HZ138551,02/02/2016 07:30:00 PM,033XX W 66TH ST,820,THEFT,$500 AND UNDER,ALLEY,False,False,...,15.0,66.0,6,1155240.0,1860661.0,2016,02/10/2018 03:50:01 PM,41.773455,-87.706480,"(41.773455295, -87.706480471)"


In [54]:
#Creating another dataframe with columns of interest
chicago_crime_df1 = chicago_crime_df[['BLOCK','PRIMARY_TYPE','DESCRIPTION','LOCATION_DESCRIPTION','COMMUNITY_AREA_NUMBER','YEAR','LATITUDE','LONGITUDE']]
print("Number of rows and columns in Chicago crime dataset with selected columns  is: ",chicago_crime_df1.shape)
chicago_crime_df1.head()

Number of rows and columns in Chicago crime dataset with selected columns  is:  (533, 8)


,BLOCK,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,COMMUNITY_AREA_NUMBER,YEAR,LATITUDE,LONGITUDE
0,047XX S KEDZIE AVE,THEFT,FROM BUILDING,SMALL RETAIL STORE,58.0,2004,41.807441,-87.703956
1,009XX N CENTRAL PARK AVE,THEFT,$500 AND UNDER,OTHER,23.0,2004,41.898280,-87.716406
2,043XX S WABASH AVE,THEFT,$500 AND UNDER,NURSING HOME/RETIREMENT HOME,38.0,2011,41.815933,-87.624642
3,083XX S KINGSTON AVE,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,46.0,2010,41.743665,-87.562463
4,033XX W 66TH ST,THEFT,$500 AND UNDER,ALLEY,66.0,2016,41.773455,-87.706480


In [55]:
#Removing records with missing data
chicago_crime_df1 = chicago_crime_df1.dropna()
chicago_crime_df1 = chicago_crime_df1.reset_index(drop=True)
print("Number of rows and columns in Chicago crime dataset after dropping missing value records is: ",chicago_crime_df1.shape)
chicago_crime_df1.head()

Number of rows and columns in Chicago crime dataset after dropping missing value records is:  (487, 8)


,BLOCK,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,COMMUNITY_AREA_NUMBER,YEAR,LATITUDE,LONGITUDE
0,047XX S KEDZIE AVE,THEFT,FROM BUILDING,SMALL RETAIL STORE,58.0,2004,41.807441,-87.703956
1,009XX N CENTRAL PARK AVE,THEFT,$500 AND UNDER,OTHER,23.0,2004,41.898280,-87.716406
2,043XX S WABASH AVE,THEFT,$500 AND UNDER,NURSING HOME/RETIREMENT HOME,38.0,2011,41.815933,-87.624642
3,083XX S KINGSTON AVE,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,46.0,2010,41.743665,-87.562463
4,033XX W 66TH ST,THEFT,$500 AND UNDER,ALLEY,66.0,2016,41.773455,-87.706480


In [56]:
#Keeping crime data prior to 2013
chicago_crime_df1 = chicago_crime_df1[chicago_crime_df1['YEAR'] < 2013]
chicago_crime_df1 = chicago_crime_df1.reset_index(drop=True)
print("Number of rows and columns in Chicago crime dataset with years prior to 2013 is: ",chicago_crime_df1.shape)
chicago_crime_df1.head()

Number of rows and columns in Chicago crime dataset with years prior to 2013 is:  (357, 8)


,BLOCK,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,COMMUNITY_AREA_NUMBER,YEAR,LATITUDE,LONGITUDE
0,047XX S KEDZIE AVE,THEFT,FROM BUILDING,SMALL RETAIL STORE,58.0,2004,41.807441,-87.703956
1,009XX N CENTRAL PARK AVE,THEFT,$500 AND UNDER,OTHER,23.0,2004,41.898280,-87.716406
2,043XX S WABASH AVE,THEFT,$500 AND UNDER,NURSING HOME/RETIREMENT HOME,38.0,2011,41.815933,-87.624642
3,083XX S KINGSTON AVE,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,46.0,2010,41.743665,-87.562463
4,006XX W CHICAGO AVE,THEFT,OVER $500,PARKING LOT/GARAGE(NON.RESID.),24.0,2010,41.896447,-87.644939


 Retrieving and Cleaning Chicago School Data

In [57]:
# Using the link from Course5 for the Chicago Public Schools progress report dataset
chicago_school_df = pd.read_csv('https://ibm.box.com/shared/static/f9gjvj1gjmxxzycdhplzt01qtz0s7ew7.csv')
print("Number of rows and columns in Chicago school report dataset is: ",chicago_school_df.shape)
chicago_school_df.head()


Number of rows and columns in Chicago school report dataset is:  (566, 78)


,School ID,NAME_OF_SCHOOL,"Elementary, Middle, or High School",Street Address,City,State,ZIP Code,Phone Number,Link,Network Manager,...,Freshman on Track Rate %,X_COORDINATE,Y_COORDINATE,Latitude,Longitude,COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,Ward,Police District,Location
0,610038,Abraham Lincoln Elementary School,ES,615 W Kemper Pl,Chicago,IL,60614,(773) 534-5720,http://schoolreports.cps.edu/SchoolProgressRep...,Fullerton Elementary Network,...,NDA,1171699.458,1915829.428,41.924497,-87.644522,7,LINCOLN PARK,43,18,"(41.92449696, -87.64452163)"
1,610281,Adam Clayton Powell Paideia Community Academy ...,ES,7511 S South Shore Dr,Chicago,IL,60649,(773) 535-6650,http://schoolreports.cps.edu/SchoolProgressRep...,Skyway Elementary Network,...,NDA,1196129.985,1856209.466,41.760324,-87.556736,43,SOUTH SHORE,7,4,"(41.76032435, -87.55673627)"
2,610185,Adlai E Stevenson Elementary School,ES,8010 S Kostner Ave,Chicago,IL,60652,(773) 535-2280,http://schoolreports.cps.edu/SchoolProgressRep...,Midway Elementary Network,...,NDA,1148427.165,1851012.215,41.747111,-87.731702,70,ASHBURN,13,8,"(41.74711093, -87.73170248)"
3,609993,Agustin Lara Elementary Academy,ES,4619 S Wolcott Ave,Chicago,IL,60609,(773) 535-4389,http://schoolreports.cps.edu/SchoolProgressRep...,Pershing Elementary Network,...,NDA,1164504.290,1873959.199,41.809757,-87.672145,61,NEW CITY,20,9,"(41.8097569, -87.6721446)"
4,610513,Air Force Academy High School,HS,3630 S Wells St,Chicago,IL,60609,(773) 535-1590,http://schoolreports.cps.edu/SchoolProgressRep...,Southwest Side High School Network,...,91.8,1175177.622,1880745.126,41.828146,-87.632794,34,ARMOUR SQUARE,11,9,"(41.82814609, -87.63279369)"


In [58]:
#Creating another dataframe with columns of interest
chicago_school_df1 = chicago_school_df[['NAME_OF_SCHOOL','Elementary, Middle, or High School','Street Address','City','State','HEALTHY_SCHOOL_CERTIFIED','AVERAGE_STUDENT_ATTENDANCE','Rate of Misconducts (per 100 students) ','Average Teacher Attendance','Individualized Education Program Compliance Rate ','Police District','Latitude','Longitude','COMMUNITY_AREA_NUMBER','COMMUNITY_AREA_NAME']]
print("Number of rows and columns in Chicago school dataset with selected columns  is: ",chicago_school_df1.shape)
chicago_school_df1.head()

Number of rows and columns in Chicago school dataset with selected columns  is:  (566, 15)


,NAME_OF_SCHOOL,"Elementary, Middle, or High School",Street Address,City,State,HEALTHY_SCHOOL_CERTIFIED,AVERAGE_STUDENT_ATTENDANCE,Rate of Misconducts (per 100 students),Average Teacher Attendance,Individualized Education Program Compliance Rate,Police District,Latitude,Longitude,COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME
0,Abraham Lincoln Elementary School,ES,615 W Kemper Pl,Chicago,IL,Yes,96.00%,2.0,96.40%,95.80%,18,41.924497,-87.644522,7,LINCOLN PARK
1,Adam Clayton Powell Paideia Community Academy ...,ES,7511 S South Shore Dr,Chicago,IL,No,95.60%,15.7,95.30%,100.00%,4,41.760324,-87.556736,43,SOUTH SHORE
2,Adlai E Stevenson Elementary School,ES,8010 S Kostner Ave,Chicago,IL,No,95.70%,2.3,94.70%,98.30%,8,41.747111,-87.731702,70,ASHBURN
3,Agustin Lara Elementary Academy,ES,4619 S Wolcott Ave,Chicago,IL,No,95.50%,10.4,95.80%,100.00%,9,41.809757,-87.672145,61,NEW CITY
4,Air Force Academy High School,HS,3630 S Wells St,Chicago,IL,Yes,93.30%,15.6,96.90%,100.00%,9,41.828146,-87.632794,34,ARMOUR SQUARE


In [59]:
#Removing records with missing data
chicago_school_df1 = chicago_school_df1.dropna()
chicago_school_df1 = chicago_school_df1.reset_index(drop=True)
print("Number of rows and columns in Chicago school dataset after dropping missing value records is: ",chicago_school_df1.shape)
chicago_school_df1.head()

Number of rows and columns in Chicago school dataset after dropping missing value records is:  (565, 15)


,NAME_OF_SCHOOL,"Elementary, Middle, or High School",Street Address,City,State,HEALTHY_SCHOOL_CERTIFIED,AVERAGE_STUDENT_ATTENDANCE,Rate of Misconducts (per 100 students),Average Teacher Attendance,Individualized Education Program Compliance Rate,Police District,Latitude,Longitude,COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME
0,Abraham Lincoln Elementary School,ES,615 W Kemper Pl,Chicago,IL,Yes,96.00%,2.0,96.40%,95.80%,18,41.924497,-87.644522,7,LINCOLN PARK
1,Adam Clayton Powell Paideia Community Academy ...,ES,7511 S South Shore Dr,Chicago,IL,No,95.60%,15.7,95.30%,100.00%,4,41.760324,-87.556736,43,SOUTH SHORE
2,Adlai E Stevenson Elementary School,ES,8010 S Kostner Ave,Chicago,IL,No,95.70%,2.3,94.70%,98.30%,8,41.747111,-87.731702,70,ASHBURN
3,Agustin Lara Elementary Academy,ES,4619 S Wolcott Ave,Chicago,IL,No,95.50%,10.4,95.80%,100.00%,9,41.809757,-87.672145,61,NEW CITY
4,Air Force Academy High School,HS,3630 S Wells St,Chicago,IL,Yes,93.30%,15.6,96.90%,100.00%,9,41.828146,-87.632794,34,ARMOUR SQUARE


Exploration of the school data: Creating a map of the schools in Chicago School DataFrame

In [60]:
#Creating a map of the schools in Chicago School DataFrame
#Obtaining the geographical co-ordinates of Chicago
address = 'Chicago, USA'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinate of Chicago are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


The geographical coordinate of Chicago are 41.8755616, -87.6244212.


In [61]:
# Creating a map using latitude and longitude values
school_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, school, areaname in zip(chicago_school_df1['Latitude'], chicago_school_df1['Longitude'], chicago_school_df1['NAME_OF_SCHOOL'], chicago_school_df1['COMMUNITY_AREA_NAME']):
    label = '{}, {}'.format(school, areaname)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(school_map)  

school_map

Exploration of the location data - from the school data

In [62]:
#Obtaining the location data from the school dataframe
chicago_df = chicago_school_df1[['COMMUNITY_AREA_NUMBER','COMMUNITY_AREA_NAME','Latitude','Longitude']]
print("Chicago location dataframe : ",chicago_df.shape)
chicago_df.head()

Chicago location dataframe :  (565, 4)


,COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,Latitude,Longitude
0,7,LINCOLN PARK,41.924497,-87.644522
1,43,SOUTH SHORE,41.760324,-87.556736
2,70,ASHBURN,41.747111,-87.731702
3,61,NEW CITY,41.809757,-87.672145
4,34,ARMOUR SQUARE,41.828146,-87.632794


Exploration of the crime data

In [63]:
#Creating new Chicago crime dataframe with selected columns from prior Chicago crime dataframe
chicago_crime_df2 = chicago_crime_df1[['COMMUNITY_AREA_NUMBER','YEAR','PRIMARY_TYPE']]
chicago_crime_df2['COMMUNITY_AREA_NUMBER'] = chicago_crime_df2['COMMUNITY_AREA_NUMBER'].astype(int)
print(chicago_crime_df2.shape)
chicago_crime_df2.head()

(357, 3)


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,COMMUNITY_AREA_NUMBER,YEAR,PRIMARY_TYPE
0,58,2004,THEFT
1,23,2004,THEFT
2,38,2011,THEFT
3,46,2010,THEFT
4,24,2010,THEFT


In [64]:
#Calculating the counts of crime in each area for every year of data available
chicago_crime_df2 = chicago_crime_df2.groupby(['COMMUNITY_AREA_NUMBER','YEAR']).size().reset_index(name='counts')
print(chicago_crime_df2.shape)
chicago_crime_df2.head()

(257, 3)


,COMMUNITY_AREA_NUMBER,YEAR,counts
0,1,2004,2
1,1,2006,2
2,1,2007,1
3,1,2011,1
4,2,2002,1


In [65]:
#Creating list of area numbers and crime types
areanumber_list = chicago_crime_df2['COMMUNITY_AREA_NUMBER'].tolist()
areanumber_list = list(set(areanumber_list))
#crimetype_list = chicago_crime_df2.groupby('PRIMARY_TYPE')['PRIMARY_TYPE'].nunique().index.tolist()
print(areanumber_list)

[1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 14, 15, 16, 17, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 35, 37, 38, 39, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 55, 56, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 76, 77]


Feature selection

In [81]:
#Developing model to predict crime counts for a future year for any Chicago area - using areanumber = 1 here
Feature_area1 = chicago_crime_df2[chicago_crime_df2['COMMUNITY_AREA_NUMBER'] == 1]
Feature_area1.head()

X = Feature_area1['YEAR']
X = X.to_numpy(X)
X = np.reshape(X,(-1,1))
X[0:5]

array([[2004],
       [2006],
       [2007],
       [2011]])

Label selection

In [80]:
y = chicago_crime_df2[chicago_crime_df2['COMMUNITY_AREA_NUMBER'] == 1]['counts'].values
y[0:5]

array([2, 2, 1, 1])

Normalize Data

In [82]:
from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


array([[-1.17669681],
       [-0.39223227],
       [ 0.        ],
       [ 1.56892908]])

In [85]:
#Using training and splitting Linear Regression models to predict the crime counts for area 1 and printing the R2 Score

from sklearn.model_selection import train_test_split
from sklearn import linear_model
regr = linear_model.LinearRegression()
from sklearn.metrics import r2_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)
regr.fit(X_train, y_train)
y_predict_area1 = regr.predict(X_test)
r2_score_area1 = r2_score(y_predict_area1 , y_test)
coefficients_area1 = regr.coef_
intercept_area1 = regr.intercept_

print('The R2 Score for area 1 for the Linear Regressions is', r2_score_area1)
print("Coefficients: ",coefficients_area1)
print("Intercept: ",intercept_area1)

The R2 Score for area 1 for the Linear Regressions is 0.0
Coefficients:  [-0.72843136]
Intercept:  1.2857142857142858


The linear regression model can be used to predict the crime counts in specific locations for future years.

In [90]:
import math

In [91]:
X = ([[2013]])
X = preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

array([[0.]])

In [113]:
y_predict_area1_2013 = math.floor(regr.predict(X))
print("Area 1 name is: ", np.array(chicago_df[chicago_df['COMMUNITY_AREA_NUMBER'] == 1]['COMMUNITY_AREA_NAME'].values)[0])
print("Predicted crime count for area 1 for year 2013 is ", y_predict_area1_2013)


Area 1 name is:  ROGERS PARK
Predicted crime count for area 1 for year 2013 is  1


Crime counts for other locations for future years can be obtained similarly using Linear Regression.

In [115]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: GXUX3TJAEFTX5W2EKJMMPW5OR05KPCTPXSPBSHNPD412KRXT
CLIENT_SECRET:CFXD5YOTRTRFEQCIZD2XWG1JOSXDD31OZMRGYV411TDTLULJ


In [136]:
#Using Foursquare API to find police stations within 2000 meters of schools in area 1
search_query = 'Police'
radius = 2000
print("Looking for: ",search_query)
chicago_school_df_area1 = chicago_school_df1[chicago_school_df1['COMMUNITY_AREA_NUMBER'] == 1]
print("Number of schools in area 1 is: ",chicago_school_df_area1.shape[0])
NAME_OF_SCHOOL1 = chicago_school_df_area1.loc[chicago_school_df_area1.index[0],'NAME_OF_SCHOOL']
print("First school in area 1 is : ",NAME_OF_SCHOOL1)
latitude = chicago_school_df_area1.loc[chicago_school_df_area1.index[0],'Latitude']
longitude = chicago_school_df_area1.loc[chicago_school_df_area1.index[0],'Longitude']
print("Latitude for first school in area 1 is : ",latitude)
print("Longitude for first school in area 1 is : ",longitude)
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
print("Foursquare URL to get the police stations within 2000 meters of chool 1 in area 1 is : ",url)
#Send the GET Request and examine the results
results = requests.get(url).json()
print("Results of the URL:")
results
print("Get relevant part of JSON and transform it into a pandas dataframe")
# assign relevant part of JSON to venues
venues = results['response']['venues']
# tranform venues into a dataframe
venues_df = json_normalize(venues)
venues_df.columns
#filter for columns of interest
venues_df1 = venues_df[['name','location.lat','location.lng']]
print("Number of police stations near school 1 in area1: ",venues_df1.shape[0])
venues_df1.rename(columns={'location.lat': 'lat', 'location.lng': 'lng'}, inplace=True)
venues_df1.head()

Looking for:  Police
Number of schools in area 1 is:  6
First school in area 1 is :  Eugene Field Elementary School
Latitude for first school in area 1 is :  42.00966462
Longitude for first school in area 1 is :  -87.66994862
Foursquare URL to get the police stations within 2000 meters of chool 1 in area 1 is :  https://api.foursquare.com/v2/venues/search?client_id=GXUX3TJAEFTX5W2EKJMMPW5OR05KPCTPXSPBSHNPD412KRXT&client_secret=CFXD5YOTRTRFEQCIZD2XWG1JOSXDD31OZMRGYV411TDTLULJ&ll=42.00966462,-87.66994862&v=20180604&query=Police&radius=2000&limit=10
Results of the URL:
Get relevant part of JSON and transform it into a pandas dataframe
Number of police stations near school 1 in area1:  6


/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,name,lat,lng
0,Chicago Police Department 24th District,41.999212,-87.671716
1,Central City Police Station,42.011312,-87.673837
2,Bellarmine Campus Police Substation,42.003311,-87.661212
3,Evanston Police Department Outpost,42.019507,-87.679944
4,Blue Police Box Music,41.992988,-87.672895


In [137]:
#Visualizing the school and the nearby police stations in a map
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the school 1

# add a red circle marker to represent the school 1
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup=NAME_OF_SCHOOL1,
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Police stations as blue circle markers
for lat, lng, label in zip(venues_df1.lat, venues_df1.lng, venues_df1.name):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

Similar analysis can be done for other schools in the area 1 and using the predicted crime count of area 1 from above, officials can determine if the area is safe for existing schools or if more police stations are required.

In [150]:
print("Getting the crime counts in an area")
chicago_crime_df3 = chicago_crime_df1[['COMMUNITY_AREA_NUMBER','PRIMARY_TYPE']]
chicago_crime_df3 = chicago_crime_df3.groupby(['COMMUNITY_AREA_NUMBER']).size().reset_index(name='crimecounts')
print(chicago_crime_df3.shape)
chicago_crime_df3.head()


Getting the crime counts in an area
(66, 2)


,COMMUNITY_AREA_NUMBER,crimecounts
0,1.0,6
1,2.0,5
2,3.0,4
3,4.0,2
4,5.0,4


In [164]:
print("Joining the area crime counts with school data")
chicago_school_df2 = chicago_school_df1[['NAME_OF_SCHOOL','Latitude','Longitude','COMMUNITY_AREA_NUMBER','COMMUNITY_AREA_NAME']]
school_crime_data = pd.merge(chicago_school_df2, chicago_crime_df3, on=['COMMUNITY_AREA_NUMBER'], how='inner')
school_crime_data.head()

Joining the area crime counts with school data


(530, 6)

Clustering the schools based on the crime counts in the area:

In [161]:
# set number of clusters
kclusters = 4

school_grouped_clustering = school_crime_data[['crimecounts','Latitude','Longitude']]

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(school_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 2, 2, 2], dtype=int32)

In [172]:
#Adding the cluster labels to the school crime data
school_crime_data['Cluster Labels'] = kmeans.labels_
school_crime_data.head()

,NAME_OF_SCHOOL,Latitude,Longitude,COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,crimecounts,Cluster Labels
0,Abraham Lincoln Elementary School,41.924497,-87.644522,7,LINCOLN PARK,5,0
1,LaSalle Elementary Language Academy,41.913882,-87.637601,7,LINCOLN PARK,5,0
2,Lincoln Park High School,41.918304,-87.645974,7,LINCOLN PARK,5,0
3,Louisa May Alcott Elementary School,41.929846,-87.646339,7,LINCOLN PARK,5,0
4,Oscar F Mayer Elementary School,41.923177,-87.657323,7,LINCOLN PARK,5,0


In [175]:
school_sample = school_crime_data.sample(n = school_crime_data.shape[0] // 3)
print("Number of schools being clustered: ",school_sample.shape[0])
school_sample.head()

Number of schools being clustered:  176


,NAME_OF_SCHOOL,Latitude,Longitude,COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,crimecounts,Cluster Labels
339,Gerald Delgado Kanoon Elementary Magnet School,41.850776,-87.705075,30,SOUTH LAWNDALE,4,0
86,Countee Cullen Elementary School,41.700361,-87.611282,49,ROSELAND,7,2
150,Frederick Funston Elementary School,41.917621,-87.716929,22,LOGAN SQUARE,6,0
347,Little Village Elementary School,41.843686,-87.717355,30,SOUTH LAWNDALE,4,0
447,Edward Jenner Elementary Academy of the Arts,41.902058,-87.641019,8,NEAR NORTH SIDE,12,2


In [177]:
# Visulaizing a sample of school clusters based on crime counts in the area

# Re-Obtaining the geographical co-ordinates of Chicago
address = 'Chicago, USA'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinate of Chicago are {}, {}.'.format(latitude, longitude))
print("Map Clusters")
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(school_sample['Latitude'], school_sample['Longitude'], school_sample['NAME_OF_SCHOOL'], school_sample['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:5: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


The geographical coordinate of Chicago are 41.8755616, -87.6244212.
Map Clusters


In [199]:
print("Analyzing Cluster RED")
print('Average Crime Rate for Cluster 0 (RED) is', school_crime_data.loc[school_crime_data['Cluster Labels'] == 0]['crimecounts'].mean())
print('School with highest crime count in Cluster 0 is', school_crime_data['NAME_OF_SCHOOL'].loc[school_crime_data.loc[school_crime_data['Cluster Labels'] == 0]['crimecounts'].argmax()], 'with a crime count of', school_crime_data['crimecounts'].loc[school_crime_data.loc[school_crime_data['Cluster Labels'] == 0]['crimecounts'].argmax()])
print('School with lowest crime count in Cluster 0 is', school_crime_data['NAME_OF_SCHOOL'].loc[school_crime_data.loc[school_crime_data['Cluster Labels'] == 0]['crimecounts'].argmin()], 'with a crime count of', school_crime_data['crimecounts'].loc[school_crime_data.loc[school_crime_data['Cluster Labels'] == 0]['crimecounts'].argmin()])
print("Unique areas in Cluster RED:")
print(np.unique(school_crime_data.loc[school_crime_data['Cluster Labels'] == 0].COMMUNITY_AREA_NAME.values))
school_crime_data.loc[school_crime_data['Cluster Labels'] == 0].head()

Analyzing Cluster RED
Average Crime Rate for Cluster 0 (RED) is 4.028213166144201
School with highest crime count in Cluster 0 is Al Raby High School with a crime count of 6
School with lowest crime count in Cluster 0 is Charles N Holden Elementary School with a crime count of 1
Unique areas in Cluster RED:
['ALBANY PARK' 'ASHBURN' 'AVALON PARK' 'AVONDALE' 'BELMONT CRAGIN'
 'BEVERLY' 'BRIDGEPORT' 'BURNSIDE' 'CALUMET HEIGHTS' 'CHATHAM' 'CLEARING'
 'DOUGLAS' 'DUNNING' 'EAST GARFIELD PARK' 'EAST SIDE' 'EDGEWATER'
 'FOREST GLEN' 'FULLER PARK' 'GAGE PARK' 'GARFIELD RIDGE'
 'GRAND BOULEVARD' 'HEGEWISCH' 'HERMOSA' 'HYDE PARK' 'IRVING PARK'
 'JEFFERSON PARK' 'KENWOOD' 'LINCOLN PARK' 'LINCOLN SQUARE' 'LOGAN SQUARE'
 'LOOP' 'MCKINLEY PARK' 'NORTH CENTER' 'NORWOOD PARK' 'OHARE'
 'PORTAGE PARK' 'PULLMAN' 'ROGERS PARK' 'SOUTH DEERING' 'SOUTH LAWNDALE'
 'UPTOWN' 'WASHINGTON HEIGHTS' 'WEST ELSDON' 'WEST ENGLEWOOD'
 'WEST GARFIELD PARK' 'WEST LAWN' 'WEST PULLMAN' 'WEST RIDGE' 'WOODLAWN']


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  app.launch_new_instance()
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:4: FutureWarning: 
The current behaviour of 'Series.argmin' is deprecated, use 'idxmin'
instead.
The behavior of 'argmin' will be corrected to return the positional
minimum in the future. For now, use 'series.values.argmin' or
'np.argmin(np.array(values))' to get the position of the minimum
row.


,NAME_OF_SCHOOL,Latitude,Longitude,COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,crimecounts,Cluster Labels
0,Abraham Lincoln Elementary School,41.924497,-87.644522,7,LINCOLN PARK,5,0
1,LaSalle Elementary Language Academy,41.913882,-87.637601,7,LINCOLN PARK,5,0
2,Lincoln Park High School,41.918304,-87.645974,7,LINCOLN PARK,5,0
3,Louisa May Alcott Elementary School,41.929846,-87.646339,7,LINCOLN PARK,5,0
4,Oscar F Mayer Elementary School,41.923177,-87.657323,7,LINCOLN PARK,5,0


Cluster RED is spread across many areas.

In [200]:
print("Analyzing Cluster PURPLE")
print('Average Crime Rate for Cluster 1 (PURPLE) is', school_crime_data.loc[school_crime_data['Cluster Labels'] == 1]['crimecounts'].mean())
print('School with highest crime count in Cluster 1 is', school_crime_data['NAME_OF_SCHOOL'].loc[school_crime_data.loc[school_crime_data['Cluster Labels'] == 1]['crimecounts'].argmax()], 'with a crime count of', school_crime_data['crimecounts'].loc[school_crime_data.loc[school_crime_data['Cluster Labels'] == 1]['crimecounts'].argmax()])
print('School with lowest crime count in Cluster 1 is', school_crime_data['NAME_OF_SCHOOL'].loc[school_crime_data.loc[school_crime_data['Cluster Labels'] == 1]['crimecounts'].argmin()], 'with a crime count of', school_crime_data['crimecounts'].loc[school_crime_data.loc[school_crime_data['Cluster Labels'] == 1]['crimecounts'].argmin()])
print("Unique areas in Cluster PURPLE:")
print(np.unique(school_crime_data.loc[school_crime_data['Cluster Labels'] == 1].COMMUNITY_AREA_NAME.values))
school_crime_data.loc[school_crime_data['Cluster Labels'] == 1].head()

Analyzing Cluster PURPLE
Average Crime Rate for Cluster 1 (PURPLE) is 32.0
School with highest crime count in Cluster 1 is Austin Business and Entrepreneurship Academy High School with a crime count of 32
School with lowest crime count in Cluster 1 is Austin Business and Entrepreneurship Academy High School with a crime count of 32
Unique areas in Cluster PURPLE:
['AUSTIN']


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  app.launch_new_instance()
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:4: FutureWarning: 
The current behaviour of 'Series.argmin' is deprecated, use 'idxmin'
instead.
The behavior of 'argmin' will be corrected to return the positional
minimum in the future. For now, use 'series.values.argmin' or
'np.argmin(np.array(values))' to get the position of the minimum
row.


,NAME_OF_SCHOOL,Latitude,Longitude,COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,crimecounts,Cluster Labels
250,Austin Business and Entrepreneurship Academy H...,41.885205,-87.763212,25,AUSTIN,32,1
251,Austin Polytechnical Academy High School,41.885205,-87.763212,25,AUSTIN,32,1
252,Edward K Ellington Elementary School,41.885506,-87.766552,25,AUSTIN,32,1
253,Ella Flagg Young Elementary School,41.906784,-87.767179,25,AUSTIN,32,1
254,Francis Scott Key Elementary School,41.889836,-87.766335,25,AUSTIN,32,1


Cluster PURPLE is only in one area.

In [201]:
print("Analyzing Cluster BLUE")
print('Average Crime Rate for Cluster 2 (BLUE) is', school_crime_data.loc[school_crime_data['Cluster Labels'] == 2]['crimecounts'].mean())
print('School with highest crime count in Cluster 2 is', school_crime_data['NAME_OF_SCHOOL'].loc[school_crime_data.loc[school_crime_data['Cluster Labels'] == 2]['crimecounts'].argmax()], 'with a crime count of', school_crime_data['crimecounts'].loc[school_crime_data.loc[school_crime_data['Cluster Labels'] == 2]['crimecounts'].argmax()])
print('School with lowest crime count in Cluster 2 is', school_crime_data['NAME_OF_SCHOOL'].loc[school_crime_data.loc[school_crime_data['Cluster Labels'] == 2]['crimecounts'].argmin()], 'with a crime count of', school_crime_data['crimecounts'].loc[school_crime_data.loc[school_crime_data['Cluster Labels'] == 2]['crimecounts'].argmin()])
print("Unique areas in Cluster BLUE:")
print(np.unique(school_crime_data.loc[school_crime_data['Cluster Labels'] == 2].COMMUNITY_AREA_NAME.values))
school_crime_data.loc[school_crime_data['Cluster Labels'] == 2].head()

Analyzing Cluster BLUE
Average Crime Rate for Cluster 2 (BLUE) is 8.930379746835444
School with highest crime count in Cluster 2 is Andrew Jackson Elementary Language Academy with a crime count of 12
School with lowest crime count in Cluster 2 is Alex Haley Elementary Academy with a crime count of 7
Unique areas in Cluster BLUE:
['AUBURN GRESHAM' 'BRIGHTON PARK' 'CHICAGO LAWN' 'GREATER GRAND CROSSING'
 'LAKE VIEW' 'LOWER WEST SIDE' 'NEAR NORTH SIDE' 'NEAR WEST SIDE'
 'NEW CITY' 'NORTH LAWNDALE' 'ROSELAND' 'SOUTH CHICAGO' 'SOUTH SHORE'
 'WEST TOWN']


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  app.launch_new_instance()
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:4: FutureWarning: 
The current behaviour of 'Series.argmin' is deprecated, use 'idxmin'
instead.
The behavior of 'argmin' will be corrected to return the positional
minimum in the future. For now, use 'series.values.argmin' or
'np.argmin(np.array(values))' to get the position of the minimum
row.


,NAME_OF_SCHOOL,Latitude,Longitude,COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,crimecounts,Cluster Labels
7,Adam Clayton Powell Paideia Community Academy ...,41.760324,-87.556736,43,SOUTH SHORE,8,2
8,Edward A Bouchet Math & Science Academy Elemen...,41.760861,-87.576038,43,SOUTH SHORE,8,2
9,Enrico Fermi Elementary School,41.767728,-87.590390,43,SOUTH SHORE,8,2
10,High School of Leadership at South Shore,41.756194,-87.579607,43,SOUTH SHORE,8,2
11,Isabelle C O'Keeffe Elementary School,41.768616,-87.572843,43,SOUTH SHORE,8,2


Cluster BLUE is spread across 14 areas.

In [197]:
print("Analyzing Cluster GREEN")
print('Average Crime Rate for Cluster 3 (GREEN) is', school_crime_data.loc[school_crime_data['Cluster Labels'] == 3]['crimecounts'].mean())
print('School with highest crime count in Cluster 3 is', school_crime_data['NAME_OF_SCHOOL'].loc[school_crime_data.loc[school_crime_data['Cluster Labels'] == 3]['crimecounts'].argmax()], 'with a crime count of', school_crime_data['crimecounts'].loc[school_crime_data.loc[school_crime_data['Cluster Labels'] == 3]['crimecounts'].argmax()])
print('School with lowest crime count in Cluster 3 is', school_crime_data['NAME_OF_SCHOOL'].loc[school_crime_data.loc[school_crime_data['Cluster Labels'] == 3]['crimecounts'].argmin()], 'with a crime count of', school_crime_data['crimecounts'].loc[school_crime_data.loc[school_crime_data['Cluster Labels'] == 3]['crimecounts'].argmin()])
print("Unique areas in Cluster GREEN:")
print(np.unique(school_crime_data.loc[school_crime_data['Cluster Labels'] == 3].COMMUNITY_AREA_NAME.values))
school_crime_data.loc[school_crime_data['Cluster Labels'] == 3].head(100)

Analyzing Cluster GREEN
Average Crime Rate for Cluster 3 (GREEN) is 17.0
School with highest crime count in Cluster 3 is Alfred Nobel Elementary School with a crime count of 17
School with lowest crime count in Cluster 3 is Alfred Nobel Elementary School with a crime count of 17
Unique areas in Cluster GREEN:
['ENGLEWOOD' 'HUMBOLDT PARK']


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  app.launch_new_instance()
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:4: FutureWarning: 
The current behaviour of 'Series.argmin' is deprecated, use 'idxmin'
instead.
The behavior of 'argmin' will be corrected to return the positional
minimum in the future. For now, use 'series.values.argmin' or
'np.argmin(np.array(values))' to get the position of the minimum
row.


,NAME_OF_SCHOOL,Latitude,Longitude,COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,crimecounts,Cluster Labels
96,Alfred Nobel Elementary School,41.906128,-87.729933,23,HUMBOLDT PARK,17,3
97,Brian Piccolo Elementary Specialty School,41.900146,-87.731298,23,HUMBOLDT PARK,17,3
98,Daniel R Cameron Elementary School,41.903785,-87.717963,23,HUMBOLDT PARK,17,3
99,George Westinghouse High School,41.889950,-87.707344,23,HUMBOLDT PARK,17,3
100,Harriet Beecher Stowe Elementary School,41.911966,-87.713285,23,HUMBOLDT PARK,17,3
101,James Russell Lowell Elementary School,41.906547,-87.710233,23,HUMBOLDT PARK,17,3
102,Laura S Ward Elementary School,41.888450,-87.717524,23,HUMBOLDT PARK,17,3
103,Martin A Ryerson Elementary School,41.893211,-87.718833,23,HUMBOLDT PARK,17,3
104,Morton School of Excellence,41.889287,-87.704988,23,HUMBOLDT PARK,17,3
105,North-Grand High School,41.911701,-87.735252,23,HUMBOLDT PARK,17,3


Cluster GREEN is spread across two areas with same crime counts.